In [2]:
from fredapi import Fred
import warnings
warnings.filterwarnings('ignore')
from fred import Fred
import pandas as pd
import re
import time
from functools import reduce

In [3]:
fr = Fred(api_key='f13196245a409ee3db300071145818eb',response_type='dict')
all_tags=pd.DataFrame(fr.tag.tags())
tag_name=all_tags['name']
related_tag=pd.DataFrame(fr.tag.related_tags('al'))
tag=related_tag['name'].sort_values()
#tag.to_csv('tag.csv')
#tag_name.sort_values()[150:200]

In [4]:
state_list=["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware",'District of Columbia',"Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
states=[x.lower() for x in states]

In [5]:
def get_series_df(series_id,title,frequency,state):
    df=pd.DataFrame(fr.series.observations(series_id))
    df=df.loc[:,['date','value']]
    df.columns=['date',title]
    df['year']=[x.year for x in df['date']]
    if frequency=='A':
        df=df.loc[:,['year',title]]
    if frequency=='Q':
        df['month']=[x.month for x in df['date']]
        df=df.loc[df['month']==1,:]
        df=df.loc[:,['year',title]]
    if frequency=='M':
        df['month']=[x.month for x in df['date']]
        df=df.loc[df['month']==1,:]
        df=df.loc[:,['year',title]] 
        '''
        #Return year data
        df=df.loc[df['month']==1,:]
        df=df.loc[:,['year',title]]
        '''
    df['state']=state
    df=df.loc[df['year']>=1992,:]
    return df

In [24]:
tag_list=['wholesale','utilities','natural resources',
          'advanced degree','air travel','arts',
          'benefits','business','capital',
         'commuting time','consumer','corporate',
         'credits','deposits','education',
         'energy','equipment','financial']

In [29]:
import re
temp_tag_df_list=[]
tag_df_list=[]
divide_count=1
for tag in tag_list:
    state_count=0
    for state in states:
        temp_search=state+';'+tag
        temp_df=pd.DataFrame(fr.tag.series(temp_search,order_by='popularity'))
        if temp_df.shape[0]==0:
            continue
        temp_df['state']=state
        temp_df['m_id']=[re.sub(state.upper(),'',x) for x in temp_df['id'] ]
        temp_df=temp_df.loc[~temp_df['title'].str.contains(
                'Gross Domestic Product|All Employees|Consumer Price Index|Personal Consumption|Number of Identified Exporters'),:]
        if state_count==0:
            state_id_df=temp_df
        else:
            state_id_df=state_id_df.append(temp_df)
        state_count+=1
    if divide_count%3!=0:    
        temp_tag_df_list.append(state_id_df)
    else:
        temp_tag_df_list.append(state_id_df)
        tag_df_list.append(temp_tag_df_list)
        temp_tag_df_list=[]
    divide_count+=1

In [19]:
test=test.loc[~test['title'].str.contains('County'),:]

In [28]:
temp_df.shape[0]

0

In [30]:
scrapy_data_list=[]
tag_count=0
for temp_df_list in tag_df_list:
    for df in temp_df_list:
        print(tag_list[tag_count])
        tag_count+=1
        get_id=df['m_id'].value_counts()
        get_id=get_id[get_id>=30]
        id_list=list(get_id.index)
        temp_df=df.loc[df['m_id'].isin(id_list)]
        print(temp_df.shape)
        scrapy_data_list.append(temp_df)
scrapy_final = reduce(lambda left,right: left.append(right),scrapy_data_list)
scrapy_final=scrapy_final.reset_index(drop=True)

wholesale
(102, 18)
utilities
(177, 18)
natural resources
(0, 18)
advanced degree
(50, 18)
air travel
(0, 18)
arts
(99, 18)
benefits
(102, 18)
business
(49, 18)
capital
(101, 18)
commuting time
(0, 18)
consumer
(51, 18)
corporate
(87, 18)
credits
(0, 18)
deposits
(570, 18)
education
(252, 18)
energy
(0, 18)
equipment
(0, 18)
financial
(51, 18)


In [ ]:
s

In [87]:
scrapy_final

,frequency,frequency_short,group_popularity,id,last_updated,notes,observation_end,observation_start,popularity,realtime_end,realtime_start,seasonal_adjustment,seasonal_adjustment_short,title,units,units_short,state,m_id
0,Quarterly,Q,1,ALEWHO,2018-09-26 14:14:40,Industry based on North American Industry Clas...,2018-04-01,1998-01-01,1,2018-11-14,2018-11-14,Seasonally Adjusted Annual Rate,SAAR,Wholesale Trade Earnings in Alabama,Thousands of Dollars,Thous. of $,al,EWHO
1,Quarterly,Q,1,ALWWHO,2018-09-26 14:14:39,Industry based on North American Industry Clas...,2018-04-01,1998-01-01,1,2018-11-14,2018-11-14,Seasonally Adjusted Annual Rate,SAAR,Wholesale Trade Wages and Salaries in Alabama,Thousands of Dollars,Thous. of $,al,WWHO
2,Quarterly,Q,0,AKWWHO,2018-09-26 14:15:49,Industry based on North American Industry Clas...,2018-04-01,1998-01-01,0,2018-11-14,2018-11-14,Seasonally Adjusted Annual Rate,SAAR,Wholesale Trade Wages and Salaries in Alaska,Thousands of Dollars,Thous. of $,ak,WWHO
3,Quarterly,Q,1,AKEWHO,2018-09-26 14:14:43,Industry based on North American Industry Clas...,2018-04-01,1998-01-01,1,2018-11-14,2018-11-14,Seasonally Adjusted Annual Rate,SAAR,Wholesale Trade Earnings in Alaska,Thousands of Dollars,Thous. of $,ak,EWHO
4,Quarterly,Q,1,AZWWHO,2018-09-26 14:14:32,Industry based on North American Industry Clas...,2018-04-01,1998-01-01,1,2018-11-14,2018-11-14,Seasonally Adjusted Annual Rate,SAAR,Wholesale Trade Wages and Salaries in Arizona,Thousands of Dollars,Thous. of $,az,WWHO
5,Quarterly,Q,1,AZEWHO,2018-09-26 14:14:33,Industry based on North American Industry Clas...,2018-04-01,1998-01-01,1,2018-11-14,2018-11-14,Seasonally Adjusted Annual Rate,SAAR,Wholesale Trade Earnings in Arizona,Thousands of Dollars,Thous. of $,az,EWHO
6,Quarterly,Q,0,ARWWHO,2018-09-26 14:15:48,Industry based on North American Industry Clas...,2018-04-01,1998-01-01,0,2018-11-14,2018-11-14,Seasonally Adjusted Annual Rate,SAAR,Wholesale Trade Wages and Salaries in Arkansas,Thousands of Dollars,Thous. of $,ar,WWHO
7,Quarterly,Q,1,AREWHO,2018-09-26 14:14:38,Industry based on North American Industry Clas...,2018-04-01,1998-01-01,1,2018-11-14,2018-11-14,Seasonally Adjusted Annual Rate,SAAR,Wholesale Trade Earnings in Arkansas,Thousands of Dollars,Thous. of $,ar,EWHO
8,Quarterly,Q,1,CAWWHO,2018-09-26 14:14:30,Industry based on North American Industry Clas...,2018-04-01,1998-01-01,1,2018-11-14,2018-11-14,Seasonally Adjusted Annual Rate,SAAR,Wholesale Trade Wages and Salaries in California,Thousands of Dollars,Thous. of $,ca,WWHO
9,Quarterly,Q,1,CAEWHO,2018-09-26 14:14:31,Industry based on North American Industry Clas...,2018-04-01,1998-01-01,1,2018-11-14,2018-11-14,Seasonally Adjusted Annual Rate,SAAR,Wholesale Trade Earnings in California,Thousands of Dollars,Thous. of $,ca,EWHO


In [31]:


def merge_id_df(df):
    df=df.reset_index(drop=True)
    state_count=0
    for i in range(df.shape[0]):
        temp_id=df.loc[i,'id']
        temp_df=get_series_df(temp_id,df.loc[i,'m_id'],df.loc[i,'frequency_short'],df.loc[i,'state'])
        if state_count==0:
            state_df=temp_df
        else:
            state_df=state_df.append(temp_df)
        state_count=state_count+1   
    #print(state_df.head(2))
    state_df.columns=['year',df.loc[i,'title'],'state']
    return state_df
grouped=scrapy_final.groupby('m_id')
group_count=0
for name, group in grouped:
    if group_count==0:
        final_df=merge_id_df(group)
    else:
        temp_df=merge_id_df(group)
        temp_df=temp_df.drop_duplicates()
        final_df=pd.merge(final_df,temp_df,on=['state','year'],how='outer')
        final_df=final_df.drop_duplicates()
    group_count+=1
final_df.columns=[re.sub('for Wyoming|in Wyoming','',x) for x in final_df.columns]
#final_df.dropna(axis=1,thresh=1200)

        
    

In [33]:
final_df.columns

Index(['year',
       'State Government Tax Collections, Occupation and Business Licenses ',
       'state', 'CredAbility Consumer Distress Index  (DISCONTINUED)',
       'State Government Tax Collections, Corporation Net Income Taxes in Wisconsin',
       'State Government Tax Collections, Corporations in General Licenses ',
       'Arts, Entertainment and Recreation Earnings ',
       'Educational Services Earnings ', 'Utilities Earnings ',
       'Wholesale Trade Earnings ', 'High School Graduate or Higher ',
       'Bachelor's Degree or Higher ',
       'People 25 Years and Over Who Have Completed an Advanced Degree ',
       'State Unemployment Benefits ',
       'Transfers less State Unemployment Benefits  (DISCONTINUED)',
       'Quarterly Weighted Average of Total Equity Capital for Commercial Banks ',
       'Total Equity Capital for Commercial Banks ',
       'State Government Tax Collections, Public Utilities - Selective Sales Taxes ',
       'State Government Tax Collection

In [130]:
print(final_df.shape)
final_df_name=('_').join(tag_list)
final_df.to_csv(final_df_name+'.csv',index=False)

(1377, 8)


In [131]:
final_df_name

'wholesale_utilities_natural resources'

In [115]:
state_df

NameError: name 'state_df' is not defined

In [115]:
state_df

NameError: name 'state_df' is not defined

In [355]:
final_df.shape

(1404, 32)

In [185]:
temp_df=pd.DataFrame(fr.tag.series('ca;sales',order_by='popularity'))
temp_df=temp_df.loc[:,['id','title']]
temp_df['id']=[re.sub('CA','',x) for x in temp_df['id'] ]
temp_df1=pd.DataFrame(fr.tag.series('ak;sales',order_by='popularity'))
temp_df1['id']=[re.sub('AK','',x) for x in temp_df1['id'] ]
m_df=pd.merge(temp_df,temp_df1,on='id',how='inner')

In [64]:
'''
total_df_list=[]
for tag in tag_list:
    state_count=0
    for state in states:
        temp_search=state+';'+tag
        #print(temp_search)
        temp_df=pd.DataFrame(fr.tag.series(temp_search,order_by='popularity'))
        temp_df=temp_df.loc[:,['id']]
        temp_df['m_id']=[re.sub(state.upper(),'',x) for x in temp_df['id'] ]
        if state_count==0:
            final_df=temp_df
        else:
            final_df=pd.merge(final_df,temp_df,on='m_id',how='inner')
        state_count+=1
    total_df_list.append(final_df)
'''


'''
albama=fr.category.series('27282',order_by='popularity',sort_order='desc')
alska=fr.category.series('27286',order_by='popularity',sort_order='desc')
#help(fr.category.series)
albama=pd.DataFrame(albama)
alska=pd.DataFrame(alska)
albama=albama.loc[:,['id','title']]
alska=alska.loc[:,['id','title']]
import re
albama['title']=[re.sub('for Alabama|in Alabama','',x) for x in albama['title'] ]
alska['title']=[re.sub('for California|in California','',x) for x in alska['title'] ]
albama['id_me']=[re.sub('AL','',x) for x in albama['id'] ]
alska['id_me']=[re.sub('CA','',x) for x in alska['id'] ]
'''

"\nalbama=fr.category.series('27282',order_by='popularity',sort_order='desc')\nalska=fr.category.series('27286',order_by='popularity',sort_order='desc')\n#help(fr.category.series)\nalbama=pd.DataFrame(albama)\nalska=pd.DataFrame(alska)\nalbama=albama.loc[:,['id','title']]\nalska=alska.loc[:,['id','title']]\nimport re\nalbama['title']=[re.sub('for Alabama|in Alabama','',x) for x in albama['title'] ]\nalska['title']=[re.sub('for California|in California','',x) for x in alska['title'] ]\nalbama['id_me']=[re.sub('AL','',x) for x in albama['id'] ]\nalska['id_me']=[re.sub('CA','',x) for x in alska['id'] ]\n"

In [364]:
from census import Census


https://api.census.gov/data/2017/acs/acs1?get=B00001_001E,NAME&for=state:*&time=2014*&key=879116b83035eef6bc66700515158f25b5f38c57

In [365]:
c = Census("879116b83035eef6bc66700515158f25b5f38c57")

In [ ]:
https://api.census.gov/data/timeseries/bds/firms?get=estabs,sic1&for=state:*&time=2014&key=879116b83035eef6bc66700515158f25b5f38c57

In [402]:
html='''https://api.census.gov/data/timeseries/bds/firms?get=estabs,sic1&for=state:*&time=2014&key=879116b83035eef6bc66700515158f25b5f38c57'''

In [367]:
import requests

In [400]:
response=requests.get(html)

In [372]:
import pandas as pd
import cenpy as cen
import pysal

In [376]:
datasets = list(cen.explorer.available(verbose=True).items())
datasets=pd.DataFrame(datasets)

In [442]:
'a' in 'abc'

True

In [2]:
'''
for i in datasets.iloc[:,1]:
    temp_str=i.lower()
    if 'business' in temp_str:
        print(temp_str)
'''

"\nfor i in datasets.iloc[:,1]:\n    temp_str=i.lower()\n    if 'business' in temp_str:\n        print(temp_str)\n"

In [ ]:
for i in datasets.iloc[:,0]:
    if i[0:3]=='BDS':
        print(i)
dataset = 'BDSFirmsTimeSeries'
con = cen.base.Connection(dataset)
con.geographies['fips']
con.query(cols=['estabs'],geo_unit='state',apikey='879116b83035eef6bc66700515158f25b5f38c57')
state_category=pd.DataFrame(fr.category.children('27281'))
state_category=state_category.loc[state_category['name'].isin(state_list),:]
state_category.reset_index(drop=True)